In [59]:
import Pkg
Pkg.activate("..")

# these lines are required only for local installations
Pkg.develop(path="../../scp_traj_opt/")
Pkg.precompile()

Pkg.add("JuMP")
Pkg.build("JuMP")

using SCPToolbox
using JuMP
using ECOS
using SCS
using Ipopt
using Printf

  Activating project at `~/Documents/SCPToolbox_tutorial`
   Resolving package versions...
  No Changes to `~/Documents/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/Documents/SCPToolbox_tutorial/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Documents/SCPToolbox_tutorial/Project.toml`
  No Changes to `~/Documents/SCPToolbox_tutorial/Manifest.toml`


In [80]:
# ..:: Construct Subproblem ::..
function construct_subpbm(params)
    
    # Construct the subproblem
    pbm = ConicProgram(params;solver = params["solver"],solver_options = params["opts"])
    
    # Create variables
    z = @new_variable(pbm, 2, "z")
    ν = @new_variable(pbm, "ν")
    η = @new_variable(pbm,"η")
    μ = @new_variable(pbm,"μ")  
    
    ### (Point out usage of dictionary pars inside constraint constructor)
    
    # Linearized nonconvex constraint
    @add_constraint(
        pbm, ZERO, "my-ncvx", (z,ν),
        begin
            local z,ν = arg
            cst["A1"]*z .+ cst["b1"] .+ ν
        end)
    
    # Hyperplane constraint
    @add_constraint(
        pbm, ZERO, "my-hypln", (z,),
        begin
            local z, = arg
            cst["A2"]*z .+ cst["b2"]
        end)
    
    # Halfspace constraint
    @add_constraint(
        pbm, NONPOS, "my-hlfspace", (z,),
        begin
            local z, = arg
            cst["A3"]*z .+ cst["b3"]
        end)
        
    # Trust-region penalty constraint
    @add_constraint(
       pbm, SOC, "my-soc", (z,η),
        begin
            local z,η = arg
            vcat(η,z .- cst["z̄"])
        end
    )
    
    # Virtual buffer penalty constraint
    @add_constraint(
       pbm, SOC, "my-soc2", (ν,μ),
        begin
            local ν,μ = arg
            vcat(μ,ν)
        end
    )
    
    
    # Cost function
    @add_cost(pbm, (η,μ,z),
        begin
            local η,μ,z = arg
            z[2] .+ cst["wtr"]*η .+ cst["wvb"]*μ
        end
    )
    
    return pbm,z,η,μ
    
end

;

In [81]:
# ..:: Initialize ::..

# Construct problem parameters
params = Dict()
params["solver"] = ECOS
params["opts"]   = Dict("verbose"=>0,"abstol"=>1e-8)
# params["opts"] = Dict("print_level"=>0)
# params["opts"] = Dict("verbose"=>0)

# Initial reference and constraint data
params["z̄"] = zeros(2);
params["A1"] = Matrix(transpose([-2*params["z̄"][1];1.0]))
params["b1"] = params["z̄"][1]^2
params["A2"] = Matrix(transpose([0.1;1]))
params["b2"] = -0.6
params["A3"] = Matrix(transpose([1.0;0.0]))
params["b3"] = -0.85

iter_max = 30; iter_conv = copy(iter_max)
ϵ_conv = 1e-8
params["wtr"] = 10.0
params["wvb"] = 10000.0
z_hist = [zeros(2) for k = 1:iter_max]
μ_hist = zeros(iter_max)
η_hist = zeros(iter_max)
cost_hist = zeros(iter_max)
;

In [82]:

# ..:: SCP Iterations ::..

k = 1 # Iteration counter
while true
    
    # Construct subproblem
    pbm,z,η,μ = construct_subpbm(params)
    
    # Solve subproblem
    exit_flag = solve!(pbm)
   
    # Save history of the subproblem
    z_hist[k] .= value(z)    
    η_hist[k] = value(η)[1]
    μ_hist[k] = value(μ)[1]
    cost_hist[k] = z_hist[k][2] 

    # Check stopping criterion
    bool_conv = max(η_hist[k],μ_hist[k]) <= ϵ_conv
    
    @printf("Iteration: %d | %s | Trust region: %7.1e | Virtual buffer: %7.1e | Cost: %.3f |\n",k,string(exit_flag),η_hist[k],abs(μ_hist[k]),cost_hist[k])
        
    # Stop at maximum iterations
    if (k == iter_max) || bool_conv
        iter_conv = k        
        break
    else
    
        # Update next reference trajectory as solution
        params["z̄"] = z_hist[k]

        # Create linearized constraint values
        params["A1"] = Matrix(transpose([-2*params["z̄"][1];1.0]))
        params["b1"] = params["z̄"][1]^2    
        
        k += 1
    end
end


Iteration: 1 | OPTIMAL | Trust region: 9.9e-01 | Virtual buffer: 5.1e-01 | Cost: 0.515 |
Iteration: 2 | OPTIMAL | Trust region: 1.2e-01 | Virtual buffer: 6.2e-14 | Cost: 0.527 |
Iteration: 3 | OPTIMAL | Trust region: 8.5e-03 | Virtual buffer: 1.9e-14 | Cost: 0.527 |
Iteration: 4 | OPTIMAL | Trust region: 4.6e-05 | Virtual buffer: 1.0e-12 | Cost: 0.527 |
Iteration: 5 | OPTIMAL | Trust region: 6.5e-13 | Virtual buffer: 2.4e-14 | Cost: 0.527 |


In [83]:
print("\n\nOptimal solution: $(z_hist[iter_conv])")
;



Optimal solution: [0.7262087349048842, 0.5273791267502363]

In [84]:
# define JuMP model and NLP solver
model = Model(Ipopt.Optimizer)

# declare variables
@variable(model,z[1:2])

# nonconvex quadratic equality constraint
@NLconstraint(model,z[2]-z[1]^2 == 0)

# convex constraints
@constraint(model,params["A2"]*z .+ params["b2"] .== 0)
@constraint(model,params["A3"]*z .+ params["b3"] .≤ 0)
@objective(model,Min,z[2])

# initial guess to NLP solver
set_start_value.(all_variables(model),ones(2))

# parse and call to solver
optimize!(model)

print("\n\nOptimal solution: $(JuMP.value.(z))")
;

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        1
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0000000e+00 5.00e-01 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  